In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
train=pd.read_csv('/home/aistudio/data/data18783/train.csv')

In [ ]:
train.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [ ]:
test=pd.read_csv('/home/aistudio/data/data18783/test.csv')

In [ ]:
test.columns

Index(['id', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [ ]:
train = pd.concat([train, test], ignore_index=True, sort=False)

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
train=reduce_mem_usage(train)

Mem. usage decreased to 3971.28 Mb (46.4% reduction)


In [ ]:
train.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C16,C17,C18,C19,C20,C21,time,day,user,clickcounts
0,1.000009e+18,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,50,1722,0,35,-1,79,2014-10-21,21,a99f214a_ddd2926e_44956a24,27.0
1,1.000017e+19,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,50,1722,0,35,100084,79,2014-10-21,21,a99f214a_96809ac8_711ee120,5.0
2,1.000037e+19,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,50,1722,0,35,100084,79,2014-10-21,21,a99f214a_b3cf8def_8a4875bd,2.0
3,1.000064e+19,0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,50,1722,0,35,100084,79,2014-10-21,21,a99f214a_e8275b8f_6332421a,1.0
4,1.000068e+19,0,0,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,50,2161,0,35,-1,157,2014-10-21,21,a99f214a_9644d0bf_779d90c2,NaN


In [5]:

train['time']=pd.to_datetime(train['hour'],format='%y%m%d%H')
train['day']=train['time'].dt.day
train['hour']=train['time'].dt.hour

train['user']=train['device_id'].astype(str)+ '_' + train['device_ip'].astype(str)+ '_' +train['device_model'].astype(str)
train['app']=train['app_id'].astype(str)+ '_' + train['app_domain'].astype(str)+ '_' +train['app_category'].astype(str)
train['site']=train['site_id'].astype(str)+ '_' + train['site_domain'].astype(str)+ '_' +train['site_category'].astype(str)




In [ ]:
#选了最近三天做训练集
matrix=train[(train.day==31)|(train.day==30)|(train.day==29)|(train.day==28)]
del train,test
gc.collect()

173

In [7]:
#喜闻乐见的count特征
matrix['device_ip_count']=matrix.groupby(['device_ip'])['device_ip'].transform('count')
matrix['device_id_count']=matrix.groupby(['device_id'])['device_id'].transform('count')
matrix['hourly_device_ip_count']=matrix.groupby(['device_ip','hour'])['device_ip'].transform('count')
matrix['hourly_device_id_count']=matrix.groupby(['device_id','hour'])['device_id'].transform('count')

matrix['user_count']=matrix.groupby(['user'])['user'].transform('count')
matrix['hourly_user_count']=matrix.groupby(['user','hour'])['user'].transform('count')
matrix['hourly_count']=matrix.groupby(['hour'])['hour'].transform('count')

matrix['app_count']=matrix.groupby(['app'])['app'].transform('count')
matrix['site_count']=matrix.groupby(['site'])['site'].transform('count')

matrix['hourly_app_count']=matrix.groupby(['app','hour'])['app'].transform('count')
matrix['hourly_site_count']=matrix.groupby(['site','hour'])['site'].transform('count')


matrix['C14_count']=matrix.groupby(['C14'])['C14'].transform('count')
matrix['C21_count']=matrix.groupby(['C21'])['C21'].transform('count')
matrix['C19_count']=matrix.groupby(['C19'])['C19'].transform('count')
matrix['C20_count']=matrix.groupby(['C20'])['C20'].transform('count')


"\nmatrix['id_count']=matrix.groupby(['site_id','app_id'])['site_id'].transform('count')\nmatrix['cat_count']=matrix.groupby(['site_category','app_category'])['site_category'].transform('count')\nmatrix['domain_count']=matrix.groupby(['site_domain','app_domain'])['site_domain'].transform('count')\nmatrix['hourly_id_count']=matrix.groupby(['site_id','app_id','hour'])['site_id'].transform('count')\n\nmatrix['hourly_cat_count']=matrix.groupby(['site_category','app_category','hour'])['site_category'].transform('count')\nmatrix['hourly_domain_count']=matrix.groupby(['site_domain','app_domain','hour'])['site_domain'].transform('count')\n\nmatrix['id_model_count']=matrix.groupby(['app_id','device_model'])['app_id'].transform('count')\nmatrix['id_mid_count']=matrix.groupby(['app_id','device_id'])['app_id'].transform('count')\nmatrix['sid_model_count']=matrix.groupby(['site_id','device_model'])['site_id'].transform('count')\nmatrix['site_domain_count']=matrix.groupby(['site_domain','C20'])['C20

In [8]:
#以下加入w2v特征
app_id_em=pd.read_csv('/home/aistudio/data/data18783/app_id_em.csv')
site_domain_em=pd.read_csv('/home/aistudio/data/data18783/site_domain_em.csv')
site_id_em=pd.read_csv('/home/aistudio/data/data18783/site_id_em.csv')


In [9]:

matrix=matrix.merge(app_id_em, on='app_id', how='left')
matrix=matrix.merge(site_id_em, on='site_id', how='left')
matrix=matrix.merge(site_domain_em, on='site_domain', how='left')


In [10]:
matrix.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'time', 'day', 'user',
       'app', 'site', 'clickcopy', 'device_ip_count', 'device_id_count',
       'hourly_device_ip_count', 'hourly_device_id_count', 'user_count',
       'hourly_user_count', 'hourly_count', 'app_count', 'site_count',
       'hourly_app_count', 'hourly_site_count', 'app_idW0', 'app_idW1',
       'app_idW2', 'app_idW3', 'app_idW4', 'app_idW5', 'site_idW0',
       'site_idW1', 'site_idW2', 'site_idW3', 'site_idW4', 'site_idW5',
       'site_domainW0', 'site_domainW1', 'site_domainW2', 'site_domainW3',
       'site_domainW4', 'site_domainW5'],
      dtype='object')

In [ ]:
#对分类特征进行hash处理,某种程度上相当于label-encode
D=2**20
for feat in ['hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'time',  'user',
       'app', 'site']:
    
    matrix.loc[:,feat]=matrix.loc[:][feat].apply(lambda x:abs(hash((feat+'_'+str(x))))% D)



In [ ]:
matrix.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,site_idW2,site_idW3,site_idW4,site_idW5,site_domainW0,site_domainW1,site_domainW2,site_domainW3,site_domainW4,site_domainW5
0,1.000053e+19,0.0,0,1005,0,1998,4062,5,4727,33,...,-1.803902,2.234095,0.318872,-0.787385,1.502016,-0.160808,-0.721087,2.507872,0.129629,-2.128301
1,1.000062e+18,0.0,0,1005,0,1998,4062,5,3094,194,...,-1.803902,2.234095,0.318872,-0.787385,1.502016,-0.160808,-0.721087,2.507872,0.129629,-2.128301
2,1.000067e+19,0.0,0,1002,0,3336,1392,5,4933,132,...,-0.208467,0.018273,-0.016244,0.051306,NaN,NaN,NaN,NaN,NaN,NaN
3,1.000083e+19,0.0,0,1005,0,1998,4062,5,58,33,...,-1.803902,2.234095,0.318872,-0.787385,1.502016,-0.160808,-0.721087,2.507872,0.129629,-2.128301
4,1.000158e+19,0.0,0,1005,0,848,3516,22,4933,132,...,-0.616026,-0.280881,-0.808455,0.315808,0.257943,0.107588,0.680755,0.682446,-0.484952,0.153898


In [ ]:
#测试时用

val=matrix[(matrix.day==30)] 
tra=matrix[(matrix.day==30)|(matrix.day==29)|(matrix.day==28)]
test=matrix[(matrix.day==31)]

del matrix
gc.collect()

66

In [13]:

features=[ 'hour', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
        'device_model', 'C14',
        'C16',  'C18', 'C19', 'C20', 'C21','app_idW0', 'app_idW1', 'app_idW2', 'app_idW3', 'app_idW4', 'app_idW5','site_idW0', 'site_idW1', 'site_idW2', 'site_idW3', 'site_idW4',
        'site_domainW0', 'site_domainW1', 'site_domainW2',
       'C1','C17','device_type', 'device_conn_type',
      'device_ip_count', 'device_id_count', 'hourly_device_ip_count',
       'hourly_device_id_count', 'user_count', 'hourly_user_count',
       'hourly_count', 'app_count', 'site_count', 'hourly_app_count',
       'hourly_site_count', 'user','app','site','C14_count','C21_count','C19_count','C20_count',
       'site_domainW3', 'site_domainW4', 'site_domainW5','site_idW0', 'site_idW1', 'site_idW2',
       'site_idW3', 'site_idW4', 'site_idW5']




In [ ]:
#测试时用
y_val=val['click']
x_val=val[features]
y_train=tra['click']
x_train=tra[features]
x_test=test[features]

In [16]:

import lightgbm as lgb


lgb_params = {
    'n_estimators' : 5000,
    'learning_rate' : 0.02,
    'colsample_bytree' : 0.3,
    'subsample' : 0.7,
    'subsample_freq' : 2,
    'num_leaves' : 16,
    'seed' : 99
}




lgb_model = lgb.LGBMClassifier(**lgb_params2)



In [15]:
lgb_model.fit(x_train, y_train, eval_set=[(x_train, y_train),(x_val, y_val)],  eval_metric='logloss',verbose=True,
            early_stopping_rounds=50)

In [ ]:
#生成提交用的submission
submission=pd.read_csv('/home/aistudio/data/data18783/Submission.csv')
test_predict = lgb_model2.predict_proba(x_test)[:, 1]
submission['click']=test_predict
submission.to_csv("/home/aistudio/data/data18783/submission.csv", index=False)


In [ ]:
submission.head()

,id,click
0,10000174058809263569,0.003924
1,10000182526920855428,0.141182
2,10000554139829213984,0.625298
3,10001094637809798845,0.002743
4,10001377041558670745,0.241326
